In [31]:
import pandas as pd
df = pd.read_csv('../data/avis/gen_clean_lemma_sent_noYc_.csv',
                  sep=",", index_col = 0)
df = df.join(pd.get_dummies(df.Société, prefix = 'bank_'))
df.head(2)

,Unnamed: 0,user,etoiles,n_avis,date_avis,date_experience,Société,text_total,jour_avis,mois_avis,...,bank__ZELF,bank__bcfinance.fr,bank__floapay.com,bank__franfinance.fr,bank__ips,bank__laNef,bank__manager.one,bank__moneywalkie.com,bank__propulsebyca.fr,bank__qui-est-la-banque-la-moins-chere.com
0,0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaireÇa reste une mauvaise expé...,20,10,...,0,0,0,0,0,0,0,0,0,0
1,1,Florian T,1,2.0,2023-10-19,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...,19,10,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Mélange aléatoire des lignes du jeu de données
df = df.sample(frac = 1, random_state=7)
# split entre jeu de test et d'entrainement
train_ratio = 0.8
total_rows = df.shape[0]
train_size = int(total_rows*train_ratio)
train = df[:train_size]
test =  df[train_size:]

print("total rows:",total_rows,"\n","train size:", train_size, 
      "\n","test size:",total_rows - train_size)


total rows: 105151 
 train size: 84120 
 test size: 21031


# subsampling manuel sur la base de la modalité minoritaire


Train

In [33]:
# normalizing
from sklearn.preprocessing import StandardScaler

# Insérez votre code ici
norm_train = train[['longueur_texte', "n_avis"]]
scaler = StandardScaler().fit(norm_train) 
train[['longueur_texte', "n_avis"]] = scaler.transform(train[['longueur_texte', "n_avis"]])


/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_51858/1110561227.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[['longueur_texte', "n_avis"]] = scaler.transform(train[['longueur_texte', "n_avis"]])


In [34]:

mod_etoiles = train.etoiles.value_counts().sort_values()
n_mod = mod_etoiles.min()
train1 = train[train.etoiles ==1].iloc[0:n_mod,:]
train2 = train[train.etoiles ==2].iloc[0:n_mod,:]
train3 = train[train.etoiles ==3].iloc[0:n_mod,:]
train4 = train[train.etoiles ==4].iloc[0:n_mod,:]
train5 = train[train.etoiles ==5].iloc[0:n_mod,:]

train_equal = pd.concat([train1,train2,train3,train4,train5])

# On mélange à nouveau 
train_equal = train_equal.sample(frac = 1, random_state=7)
train_equal.etoiles.value_counts()

5    2137
4    2137
2    2137
3    2137
1    2137
Name: etoiles, dtype: int64

In [35]:
# saving balanced train dataset
train_equal.to_csv("../data/avis/train_noYC_lemma_sent_equil.csv")

Test

In [36]:
test[['longueur_texte', "n_avis"]] = scaler.transform(test[['longueur_texte', "n_avis"]])

mod_etoiles = test.etoiles.value_counts().sort_values()
n_mod = mod_etoiles.min()
test1 = test[test.etoiles ==1].iloc[0:n_mod,:]
test2 = test[test.etoiles ==2].iloc[0:n_mod,:]
test3 = test[test.etoiles ==3].iloc[0:n_mod,:]
test4 = test[test.etoiles ==4].iloc[0:n_mod,:]
test5 = test[test.etoiles ==5].iloc[0:n_mod,:]

test_equal = pd.concat([test1,test2,test3,test4,test5])

# On mélange à nouveau 
test_equal = test_equal.sample(frac = 1, random_state=7)
test_equal.etoiles.value_counts()

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_51858/1170318201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['longueur_texte', "n_avis"]] = scaler.transform(test[['longueur_texte', "n_avis"]])


2    562
3    562
4    562
1    562
5    562
Name: etoiles, dtype: int64

In [37]:
# saving balanced test dataset
test_equal.to_csv("../data/avis/test_noYC_lemma_sent_equil.csv")